### Analysis of Everypol (after JSON to pd)

In [1]:
import os
import pandas as pd
import json

In [2]:
# Folder containing JSON files
json_folder = "../data/everypol_hibp"

# Initialize an empty set to collect all unique names
all_names = set()

# Iterate over JSON files to extract all unique names
for filename in os.listdir(json_folder):
    if filename.endswith(".json"):
        file_path = os.path.join(json_folder, filename)
        with open(file_path, "r") as file:
            data = json.load(file)
            all_names.update(entry["Name"] for entry in data)

# Convert the set of all names to a sorted list
all_names = sorted(all_names)

# Initialize an empty DataFrame with columns for all names and filenames
df = pd.DataFrame(columns=["Filename"] + all_names)

# Populate the DataFrame with boolean values
for filename in os.listdir(json_folder):
    if filename.endswith(".json"):
        file_path = os.path.join(json_folder, filename)
        with open(file_path, "r") as file:
            data = json.load(file)
            present_names = {entry["Name"] for entry in data}
        
        # Create a row with filename and boolean values for each name
        row = {"Filename": filename.replace(".json", "")}
        row.update({name: name in present_names for name in all_names})
        df = pd.concat([df, pd.DataFrame([row])], ignore_index=True)

In [3]:
df.head()

,Filename,000webhost,123RF,2844Breaches,500px,8fit,ABFRL,AKP,ActMobile,Acuity,...,Zacks,Zomato,Zynga,bigbasket,db8151dd,digiDirect,eThekwiniMunicipality,iMesh,ixigo,piZap
0,k.verhoeven@tweedekamer.nl,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
1,sadhvisavitrib.foole@sansad.nic.in,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
2,mariomichele.giarrusso@senato.it,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,helen.hayes.mp@parliament.uk,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,rmsingh@sansad.nic.in,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False


In [4]:
long_df = pd.melt(
    df,
    id_vars=['Filename'],
    var_name='Breach',
    value_name='Present'
)
long_df.shape

(661195, 3)

In [5]:
long_df.head()

,Filename,Breach,Present
0,k.verhoeven@tweedekamer.nl,000webhost,False
1,sadhvisavitrib.foole@sansad.nic.in,000webhost,False
2,mariomichele.giarrusso@senato.it,000webhost,False
3,helen.hayes.mp@parliament.uk,000webhost,False
4,rmsingh@sansad.nic.in,000webhost,False


In [7]:
long_df.to_csv("../data/everypol_hibp.csv", index=False)

In [10]:
breaches = pd.read_csv("../data/hipb_01_2025_breaches_data.csv")
breaches.head()

,Name,Title,Domain,BreachDate,AddedDate,ModifiedDate,PwnCount,Description,LogoPath,DataClasses,IsVerified,IsFabricated,IsSensitive,IsRetired,IsSpamList,IsMalware,IsSubscriptionFree
0,000webhost,000webhost,000webhost.com,2015-03-01,2015-10-26T23:35:45Z,2017-12-10T21:44:27Z,14936670,"In approximately March 2015, the free web host...",https://haveibeenpwned.com/Content/Images/Pwne...,"['Email addresses', 'IP addresses', 'Names', '...",True,False,False,False,False,False,False
1,123RF,123RF,123rf.com,2020-03-22,2020-11-15T00:59:50Z,2020-11-15T01:07:10Z,8661578,"In March 2020, the stock photo site <a href=""h...",https://haveibeenpwned.com/Content/Images/Pwne...,"['Email addresses', 'IP addresses', 'Names', '...",True,False,False,False,False,False,False
2,126,126,126.com,2012-01-01,2016-10-08T07:46:05Z,2016-10-08T07:46:05Z,6414191,"In approximately 2012, it's alleged that the C...",https://haveibeenpwned.com/Content/Images/Pwne...,"['Email addresses', 'Passwords']",False,False,False,False,False,False,False
3,17Media,17,17app.co,2016-04-19,2016-07-08T01:55:03Z,2016-07-08T01:55:03Z,4009640,"In April 2016, customer data obtained from the...",https://haveibeenpwned.com/Content/Images/Pwne...,"['Device information', 'Email addresses', 'IP ...",True,False,False,False,False,False,False
4,17173,17173,17173.com,2011-12-28,2018-04-28T04:53:15Z,2018-04-28T04:53:15Z,7485802,"In late 2011, <a href=""https://news.softpedia....",https://haveibeenpwned.com/Content/Images/Pwne...,"['Email addresses', 'Passwords', 'Usernames']",False,False,False,False,False,False,False


In [11]:
merged_df = long_df.merge(breaches, left_on="Breach", right_on="Name", how="left")
merged_df.head()

,Filename,Breach,Present,Name,Title,Domain,BreachDate,AddedDate,ModifiedDate,PwnCount,Description,LogoPath,DataClasses,IsVerified,IsFabricated,IsSensitive,IsRetired,IsSpamList,IsMalware,IsSubscriptionFree
0,k.verhoeven@tweedekamer.nl,000webhost,False,000webhost,000webhost,000webhost.com,2015-03-01,2015-10-26T23:35:45Z,2017-12-10T21:44:27Z,14936670,"In approximately March 2015, the free web host...",https://haveibeenpwned.com/Content/Images/Pwne...,"['Email addresses', 'IP addresses', 'Names', '...",True,False,False,False,False,False,False
1,sadhvisavitrib.foole@sansad.nic.in,000webhost,False,000webhost,000webhost,000webhost.com,2015-03-01,2015-10-26T23:35:45Z,2017-12-10T21:44:27Z,14936670,"In approximately March 2015, the free web host...",https://haveibeenpwned.com/Content/Images/Pwne...,"['Email addresses', 'IP addresses', 'Names', '...",True,False,False,False,False,False,False
2,mariomichele.giarrusso@senato.it,000webhost,False,000webhost,000webhost,000webhost.com,2015-03-01,2015-10-26T23:35:45Z,2017-12-10T21:44:27Z,14936670,"In approximately March 2015, the free web host...",https://haveibeenpwned.com/Content/Images/Pwne...,"['Email addresses', 'IP addresses', 'Names', '...",True,False,False,False,False,False,False
3,helen.hayes.mp@parliament.uk,000webhost,False,000webhost,000webhost,000webhost.com,2015-03-01,2015-10-26T23:35:45Z,2017-12-10T21:44:27Z,14936670,"In approximately March 2015, the free web host...",https://haveibeenpwned.com/Content/Images/Pwne...,"['Email addresses', 'IP addresses', 'Names', '...",True,False,False,False,False,False,False
4,rmsingh@sansad.nic.in,000webhost,False,000webhost,000webhost,000webhost.com,2015-03-01,2015-10-26T23:35:45Z,2017-12-10T21:44:27Z,14936670,"In approximately March 2015, the free web host...",https://haveibeenpwned.com/Content/Images/Pwne...,"['Email addresses', 'IP addresses', 'Names', '...",True,False,False,False,False,False,False


In [15]:
boolean_columns = merged_df.select_dtypes(include=['bool'])
grouped_bool_counts = boolean_columns.groupby(df["Filename"]).sum().reset_index()
grouped_bool_counts.describe()

,IsVerified,IsFabricated,IsSensitive,IsRetired,IsSpamList,IsMalware,IsSubscriptionFree
count,2965.0,2965.0,2965.0,2965.0,2965.0,2965.0,2965.0
mean,1.0,0.0,0.0,0.0,0.0,0.0,0.0
std,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,1.0,0.0,0.0,0.0,0.0,0.0,0.0
25%,1.0,0.0,0.0,0.0,0.0,0.0,0.0
50%,1.0,0.0,0.0,0.0,0.0,0.0,0.0
75%,1.0,0.0,0.0,0.0,0.0,0.0,0.0
max,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
grouped_bool_counts.head()

,Filename,IsVerified,IsFabricated,IsSensitive,IsRetired,IsSpamList,IsMalware,IsSubscriptionFree
0,2016kimss@gmail.com,1,0,0,0,0,0,0
1,2ashokrai@gmail.com,1,0,0,0,0,0,0
2,2yangs6288@naver.com,1,0,0,0,0,0,0
3,38ashokroad@gmail.com,1,0,0,0,0,0,0
4,5321tempia@hanmail.net,1,0,0,0,0,0,0


In [17]:
pol_dat = pd.read_csv("../data/everypol_combined_legislature_data.csv", low_memory=False)
pol_dat.head()

,id,name,sort_name,email,twitter,facebook,group,group_id,area_id,area,...,n_unique_emails,person_count_legistype,ltype,url,cc,leg_start_date,lastmod,cc3,pop2024,lastmod_year
0,35ad9676-8485-4137-9a16-50f2844f3ab2,"Adhalrao Patil, Shri Shivaji","Adhalrao Patil, Shri Shivaji",shivajirao@sansad.nic.in,NaN,NaN,Shiv Sena,SS,shirur,Shirur,...,511,541,unicameral legislature,https://cdn.rawgit.com/everypolitician/everypo...,IN,2014-05-26,1557812040,IND,1.441720e+09,2019
1,f1f0f31d-ddb8-4681-82c0-4484d0d26ee3,"Adhikari, Shri Deepak (Dev)","Adhikari, Shri Deepak (Dev)",adhikari.deepak@sansad.nic.in,idevadhikari,IamTheDev,All India Trinamool Congress,AITC,ghatal,Ghatal,...,511,541,unicameral legislature,https://cdn.rawgit.com/everypolitician/everypo...,IN,2014-05-26,1557812040,IND,1.441720e+09,2019
2,bb68e3c6-de79-4d07-ad1f-90bb9a61ded0,"Adhikari, Shri Sisir Kumar","Adhikari, Shri Sisir Kumar",sisiradhikari76@yahoo.com,NaN,NaN,All India Trinamool Congress,AITC,kanthi,Kanthi,...,511,541,unicameral legislature,https://cdn.rawgit.com/everypolitician/everypo...,IN,2014-05-26,1557812040,IND,1.441720e+09,2019
3,c9a891c3-adb6-4581-b93b-72f9c00beacc,"Adhikari, Shri Suvendu","Adhikari, Shri Suvendu",adhikari.suvendu@sansad.nic.in,NaN,NaN,All India Trinamool Congress,AITC,tamluk,Tamluk,...,511,541,unicameral legislature,https://cdn.rawgit.com/everypolitician/everypo...,IN,2014-05-26,1557812040,IND,1.441720e+09,2019
4,5c22c70d-8317-4f94-97ec-2ccadd1acdf3,"Adityanath , Shri Yogi","Adityanath , Shri Yogi",yogi.adityanath@sansad.nic.in,NaN,NaN,Bharatiya Janata Party,BJP,gorakhpur,Gorakhpur,...,511,541,unicameral legislature,https://cdn.rawgit.com/everypolitician/everypo...,IN,2014-05-26,1557812040,IND,1.441720e+09,2019


In [26]:
pol_hibp = pol_dat.merge(grouped_bool_counts, left_on="email", right_on="Filename", how="left")
pol_hibp.head()

,id,name,sort_name,email,twitter,facebook,group,group_id,area_id,area,...,pop2024,lastmod_year,Filename,IsVerified,IsFabricated,IsSensitive,IsRetired,IsSpamList,IsMalware,IsSubscriptionFree
0,35ad9676-8485-4137-9a16-50f2844f3ab2,"Adhalrao Patil, Shri Shivaji","Adhalrao Patil, Shri Shivaji",shivajirao@sansad.nic.in,NaN,NaN,Shiv Sena,SS,shirur,Shirur,...,1.441720e+09,2019,shivajirao@sansad.nic.in,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,f1f0f31d-ddb8-4681-82c0-4484d0d26ee3,"Adhikari, Shri Deepak (Dev)","Adhikari, Shri Deepak (Dev)",adhikari.deepak@sansad.nic.in,idevadhikari,IamTheDev,All India Trinamool Congress,AITC,ghatal,Ghatal,...,1.441720e+09,2019,adhikari.deepak@sansad.nic.in,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,bb68e3c6-de79-4d07-ad1f-90bb9a61ded0,"Adhikari, Shri Sisir Kumar","Adhikari, Shri Sisir Kumar",sisiradhikari76@yahoo.com,NaN,NaN,All India Trinamool Congress,AITC,kanthi,Kanthi,...,1.441720e+09,2019,sisiradhikari76@yahoo.com,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,c9a891c3-adb6-4581-b93b-72f9c00beacc,"Adhikari, Shri Suvendu","Adhikari, Shri Suvendu",adhikari.suvendu@sansad.nic.in,NaN,NaN,All India Trinamool Congress,AITC,tamluk,Tamluk,...,1.441720e+09,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5c22c70d-8317-4f94-97ec-2ccadd1acdf3,"Adityanath , Shri Yogi","Adityanath , Shri Yogi",yogi.adityanath@sansad.nic.in,NaN,NaN,Bharatiya Janata Party,BJP,gorakhpur,Gorakhpur,...,1.441720e+09,2019,yogi.adityanath@sansad.nic.in,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [27]:
pol_hibp.describe()

,row_id,leg_start_year,row_count,n_unique_emails,person_count_legistype,lastmod,pop2024,lastmod_year,IsVerified,IsFabricated,IsSensitive,IsRetired,IsSpamList,IsMalware,IsSubscriptionFree
count,25087.000000,25087.000000,25087.000000,25087.000000,25087.000000,2.508700e+04,2.435600e+04,25087.000000,6176.0,6176.0,6176.0,6176.0,6176.0,6176.0,6176.0
mean,260.044964,2011.919201,328.801252,172.194643,750.700961,1.556683e+09,6.786439e+07,2018.997050,1.0,0.0,0.0,0.0,0.0,0.0,0.0
std,164.706621,4.832221,191.351192,149.959136,536.157515,1.595232e+06,2.106131e+08,0.054232,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,1.000000,1997.000000,31.000000,31.000000,31.000000,1.542919e+09,8.034100e+04,2018.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0
25%,119.000000,2010.000000,171.000000,66.000000,290.000000,1.555978e+09,6.378654e+06,2019.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0
50%,229.000000,2013.000000,299.000000,121.000000,554.000000,1.557278e+09,2.688913e+07,2019.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0
75%,385.000000,2015.000000,432.000000,229.000000,1437.000000,1.557812e+09,6.855680e+07,2019.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0
max,749.000000,2019.000000,686.000000,646.000000,1783.000000,1.557904e+09,1.441720e+09,2019.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
columns_to_fill = ['IsVerified', 'IsFabricated', 'IsSensitive', 
                   'IsRetired', 'IsSpamList', 'IsMalware', 'IsSubscriptionFree']
pol_hibp[columns_to_fill] = pol_hibp[columns_to_fill].fillna(0)
list(pol_hibp)

['id',
 'name',
 'sort_name',
 'email',
 'twitter',
 'facebook',
 'group',
 'group_id',
 'area_id',
 'area',
 'chamber',
 'term_x',
 'start_date',
 'end_date',
 'image',
 'gender',
 'wikidata',
 'wikidata_group',
 'wikidata_area',
 'row_id',
 'country',
 'legislature',
 'term_y',
 'leg_start_year',
 'row_count',
 'n_unique_emails',
 'person_count_legistype',
 'ltype',
 'url',
 'cc',
 'leg_start_date',
 'lastmod',
 'cc3',
 'pop2024',
 'lastmod_year',
 'Filename',
 'IsVerified',
 'IsFabricated',
 'IsSensitive',
 'IsRetired',
 'IsSpamList',
 'IsMalware',
 'IsSubscriptionFree']

In [34]:
pol_hibp.groupby(['country', 'legislature', 'leg_start_year']).mean(numeric_only=True).reset_index().describe()

,leg_start_year,row_id,row_count,n_unique_emails,person_count_legistype,lastmod,pop2024,lastmod_year,IsVerified,IsFabricated,IsSensitive,IsRetired,IsSpamList,IsMalware,IsSubscriptionFree
count,114.000000,114.000000,114.000000,114.000000,114.000000,1.140000e+02,1.050000e+02,114.000000,114.000000,114.0,114.0,114.0,114.0,114.0,114.0
mean,2012.754386,348.466660,214.198545,116.231619,478.280702,1.556414e+09,3.887496e+07,2018.982456,0.210754,0.0,0.0,0.0,0.0,0.0,0.0
std,4.075746,187.531462,157.149483,109.497399,442.561200,2.479202e+06,1.431282e+08,0.131866,0.212527,0.0,0.0,0.0,0.0,0.0,0.0
min,1997.000000,1.000000,31.000000,31.000000,31.000000,1.542919e+09,8.034100e+04,2018.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
25%,2011.000000,196.500000,111.000000,51.250000,171.750000,1.555979e+09,5.262079e+06,2019.000000,0.052679,0.0,0.0,0.0,0.0,0.0,0.0
50%,2014.000000,365.500000,170.000000,73.000000,292.000000,1.557280e+09,1.031910e+07,2019.000000,0.139077,0.0,0.0,0.0,0.0,0.0,0.0
75%,2016.000000,488.750000,297.000000,142.250000,606.000000,1.557718e+09,4.040918e+07,2019.000000,0.328083,0.0,0.0,0.0,0.0,0.0,0.0
max,2019.000000,749.000000,686.000000,646.000000,1783.000000,1.557904e+09,1.441720e+09,2019.000000,0.903704,0.0,0.0,0.0,0.0,0.0,0.0
